In [ ]:
#Module imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import os
import torch.nn.functional as F

In [ ]:
# embeddings = torch.load('../data/esmvalset2.pt')

In [ ]:
dataset_folder = "../google_prot_fns/"
filters = 64
epochs = 60
batch_size = 128
protein_len = 200

### As a toy example we only select the largest 2 protein families

In [ ]:
protein_family_number = 2

In [ ]:
def read_data(name_sub_folder):
  full_data = []
  for f in os.listdir(os.path.join(dataset_folder, name_sub_folder)):
    data = pd.read_csv(os.path.join(dataset_folder, name_sub_folder, f))
    full_data.append(data)
  return pd.concat(full_data)  

In [ ]:
dataset_train = read_data("train")
dataset_train.head()

,family_id,sequence_name,family_accession,aligned_sequence,sequence
0,PRANC,V234_FOWPN/317-415,PF09372.10,FQDLSLFDLLSNEDN..IAIVYRLSDTLLEKMN...II..KTIFPN...,FQDLSLFDLLSNEDNIAIVYRLSDTLLEKMNIIKTIFPNCFRIIQN...
1,LytR_C,S5V4G6_STRC3/409-501,PF13399.6,RIAV.QVRNG.TGADGQAPVPQR.AGVIAQVL...Q....GKG.FT...,RIAVQVRNGTGADGQAPVPQRAGVIAQVLQGKGFTEAAKDTTAAAE...
2,Zn_peptidase_2,R6B6D7_9CLOT/8-223,PF04298.12,.VILMLIIIVLPLYANIKINSTYSKY.SKKQNSG..RLTGKEVAEK...,VILMLIIIVLPLYANIKINSTYSKYSKKQNSGRLTGKEVAEKILEM...
3,UvrC_HhH_N,UVRC_MYCGA/371-525,PF08459.11,KKLIKVD.K.LN..HLEVYDNSN.LFNTDK.VSAMIVFEN..NQFN...,KKLIKVDKLNHLEVYDNSNLFNTDKVSAMIVFENNQFNKKKYRKYK...
4,Sedlin_N,Q7QCA5_ANOGA/9-136,PF04628.13,IVGHNDNPIFETEFVTVN..KEAKKE.......D.......HRHL....,IVGHNDNPIFETEFVTVNKEAKKEDHRHLNQFIAHAALDLIDEHKW...


In [ ]:
dataset_test = read_data("test")
dataset_test.head()

,family_id,sequence_name,family_accession,aligned_sequence,sequence
0,GNAT_acetyltran,R6RQF6_9CLOT/17-251,PF12746.7,AFLFSGR..REVMAD....ACLQGMM..GCVYG..........TAG...,AFLFSGRREVMADACLQGMMGCVYGTAGGMDSAAAVLGDFCFLAGK...
1,MoaC,W5NKR5_LEPOC/505-640,PF01967.21,MVDVGGK.PVSRRTAAASATVLLG.EK..........AFWLV.......,MVDVGGKPVSRRTAAASATVLLGEKAFWLVKENQLAKGDALAVAQI...
2,Methyltransf_25,C0QLU8_DESAH/50-147,PF13649.6,VLDVACGT.C...D..VA...ME..AR.NQ.......T....G......,VLDVACGTCDVAMEARNQTGDAAFIIGTDFSPGMLTLGLQKLKKNR...
3,EMG1,T1G7Q2_HELRO/22-222,PF03587.14,VVLERASLESVKV..G.................KEYQLLN....CD...,VVLERASLESVKVGKEYQLLNCDRHKGIAKKFKRDISTCRPDITHQ...
4,Glyco_hydro_30C,C6VRM9_DYAFD/453-540,PF17189.4,GAVRVDVSGGLGTD...............AMVVSSYLN..TDKSLV...,GAVRVDVSGGLGTDAMVVSSYLNTDKSLVTVIVNADNQDRDISLAI...


In [ ]:
dataset_dev = read_data("dev")
dataset_dev.head()

,family_id,sequence_name,family_accession,aligned_sequence,sequence
0,zf-Tim10_DDP,N1QB11_PSEFD/15-76,PF02953.15,..RMEKKQMKDFMNMYSNLVQRCFNDCV...........TD.F......,RMEKKQMKDFMNMYSNLVQRCFNDCVTDFTSKSLQSKEEGCVMRCV...
1,DNA_primase_S,A8XA78_CAEBR/105-345,PF01896.19,FDID..LTDYDNIRNCCKEATVCPKCWKFMVLAVKILDFLLDDMFG...,FDIDLTDYDNIRNCCKEATVCPKCWKFMVLAVKILDFLLDDMFGFN...
2,Col_cuticle_N,A8XBM5_CAEBR/9-56,PF01484.17,ASAAILSGATIVGCLFFAAQIFNEVNSLYDDVMVDMDAFKVKSNIA...,ASAAILSGATIVGCLFFAAQIFNEVNSLYDDVMVDMDAFKVKSNIAWD
3,GST_C_3,W4XBU3_STRPU/120-207,PF14497.6,KD.................................KLKESLPKTVN...,KDKLKESLPKTVNPILLKFLEKALEDNPNGNGYFVGQDATMVEFVY...
4,Ada_Zn_binding,E8U5K2_DEIML/9-73,PF02805.16,DRWQAVVQRE...AAQ.DG...LFLYAVRTTGIYCRPSCPSRRPR....,DRWQAVVQREAAQDGLFLYAVRTTGIYCRPSCPSRRPRRENVTFFE...


In [ ]:
dataset_train['family_accession'].value_counts()

PF13649.6     3637
PF00560.33    1927
PF13508.7     1761
PF06580.13    1537
PF02397.16    1528
              ... 
PF17595.2        1
PF15895.5        1
PF09125.10       1
PF07407.11       1
PF14586.6        1
Name: family_accession, Length: 17929, dtype: int64

In [ ]:
all_datasets = pd.concat([dataset_train, dataset_test, dataset_dev])

In [ ]:
# Take the data from Google format to a Fasta file.
ofile = open("all_datasets_fasta.fasta", "w")

for row in all_datasets.iterrows():

    ofile.write(">" + row[1]['sequence_name'] + "\n" + row[1]['sequence'] + "\n")

    #do not forget to close it

ofile.close()all_datasetsall_datasets

In [ ]:
# classes = all_datasets['family_accession'].value_counts()[:protein_family_number].index

In [ ]:
# def return_from_dataset(dataset, classes):
#   return [dataset.loc[dataset['family_accession'].isin(classes)].reset_index(), classes]

In [ ]:
# [toy_dataset, classes] = return_from_dataset(all_datasets, classes)

In [ ]:
# toy_dataset.to_csv("../data/2_class.csv")

We reset to save memory. Above is an example for loading the data.

In [ ]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [ ]:
#Module imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import os
import torch.nn.functional as F

In [ ]:
dataset_folder = "../google_prot_fns/"
filters = 64
epochs = 60
batch_size = 128
protein_len = 200

In [ ]:
esm_model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm2_t33_650M_UR50D")
# model_data = torch.load('/data/rsg/chemistry/johnyang/home/pt_models/esm2_t33_650M_UR50D.pt')
# regression_data = torch.load('/data/rsg/chemistry/johnyang/home/pt_models/esm2_t33_650M_UR50D-contact-regression.pt')

Using cache found in /home/ubuntu/.cache/torch/hub/facebookresearch_esm_main


In [ ]:
import itertools
import os
from typing import Sequence, Tuple, List, Union
import pickle
import re
import shutil
import torch
from pathlib import Path
RawMSA = Sequence[Tuple[str, str]]
class FastaBatchedDataset(object):
    def __init__(self, sequence_labels, sequence_strs):
        self.sequence_labels = list(sequence_labels)
        self.sequence_strs = list(sequence_strs)
    @classmethod
    def from_file(cls, fasta_file):
        sequence_labels, sequence_strs = [], []
        cur_seq_label = None
        buf = []
        def _flush_current_seq():
            nonlocal cur_seq_label, buf
            if cur_seq_label is None:
                return
            sequence_labels.append(cur_seq_label)
            sequence_strs.append("".join(buf))
            cur_seq_label = None
            buf = []
        with open(fasta_file, "r") as infile:
            for line_idx, line in enumerate(infile):
                if line.startswith(">"):  # label line
                    _flush_current_seq()
                    line = line[1:].strip()
                    if len(line) > 0:
                        cur_seq_label = line
                    else:
                        cur_seq_label = f"seqnum{line_idx:09d}"
                else:  # sequence line
                    buf.append(line.strip())
        _flush_current_seq()
        assert len(set(sequence_labels)) == len(
            sequence_labels
        ), "Found duplicate sequence labels"
        return cls(sequence_labels, sequence_strs)
    def __len__(self):
        return len(self.sequence_labels)
    def __getitem__(self, idx):
        return self.sequence_labels[idx], self.sequence_strs[idx]
    def get_batch_indices(self, toks_per_batch, extra_toks_per_seq=0):
        sizes = [(len(s), i) for i, s in enumerate(self.sequence_strs)]
        sizes.sort()
        batches = []
        buf = []
        max_len = 0
        def _flush_current_buf():
            nonlocal max_len, buf
            if len(buf) == 0:
                return
            batches.append(buf)
            buf = []
            max_len = 0
        for sz, i in sizes:
            sz += extra_toks_per_seq
            if max(sz, max_len) * (len(buf) + 1) > toks_per_batch:
                _flush_current_buf()
            max_len = max(max_len, sz)
            buf.append(i)
        _flush_current_buf()
        return batches

In [ ]:
# esm_model, alphabet = load_model_and_alphabet_core("esm2_t33_650M_UR50D", model_data, regression_data)
batch_converter = alphabet.get_batch_converter()
esm_model.to('cuda')
esm_model.eval()
print('done')

done


In [ ]:
import pathlib

In [ ]:
model = esm_model
model.eval()

toks_per_batch = 100000
fasta_file = './all_datasets_fasta.fasta'
output_dir = '../data/results'
output_file = 'run1'

model = model.cuda()
print("Transferred model to GPU")

dataset = FastaBatchedDataset.from_file('./all_datasets_fasta.fasta')
batches = dataset.get_batch_indices(toks_per_batch, extra_toks_per_seq=1)
data_loader = torch.utils.data.DataLoader(
    dataset, collate_fn=alphabet.get_batch_converter(), batch_sampler=batches
)
print(f"Read {fasta_file} with {len(dataset)} sequences")

output_dir = pathlib.Path(output_dir)
output_dir.mkdir(parents=True, exist_ok=True)
return_contacts = False

assert all(-(model.num_layers + 1) <= i <= model.num_layers for i in [33]) #[33] is the last layer of the thing
repr_layers = [(i + model.num_layers + 1) % (model.num_layers + 1) for i in [33]]

with torch.no_grad():
    for batch_idx, (labels, strs, toks) in enumerate(data_loader):
        print(
            f"Processing {batch_idx + 1} of {len(batches)} batches ({toks.size(0)} sequences)"
        )
        if torch.cuda.is_available():
            toks = toks.to(device="cuda", non_blocking=True)

        out = model(toks, repr_layers=repr_layers, return_contacts=return_contacts)

        logits = out["logits"].to(device="cpu")
        representations = {
            layer: t.to(device="cpu") for layer, t in out["representations"].items()
        }
        
        if return_contacts:
            contacts = out["contacts"].to(device="cpu")

        for i, label in enumerate(labels):
            output_file = output_dir / f"{label}.pt"
            output_file.parent.mkdir(parents=True, exist_ok=True)
            # assert os.path.is_dir(args.output_dir), f"Output directory {args.output_dir} does not exist"
            result = {"label": label}
            truncate_len = min(200, len(strs[i])) #TODO: Make that better, should be protien_len
            # Call clone on tensors to ensure tensors are not views into a larger representation
            # See https://github.com/pytorch/pytorch/issues/1995
            
            if (1): # "per_tok" in args.include:
                result["representations"] = {
                    layer: t[i, 1 : truncate_len + 1].clone()
                    for layer, t in representations.items()
                }
            # if "mean" in args.include:
            #     result["mean_representations"] = {
            #         layer: t[i, 1 : truncate_len + 1].mean(0).clone()
            #         for layer, t in representations.items()
            #     }
            # if "bos" in args.include:
            #     result["bos_representations"] = {
            #         layer: t[i, 0].clone() for layer, t in representations.items()
            #     }
            # if return_contacts:
            #     result["contacts"] = contacts[i, : truncate_len, : truncate_len].clone()
            # print(f"Saving {args.output_file}")
            torch.save(
                result,
                output_file,
            )
            assert os.path.isfile(output_file), f"Output file {output_file} does not exist"


Transferred model to GPU
Read ./all_datasets_fasta.fasta with 1339083 sequences
Processing 1 of 2096 batches (4761 sequences)
Processing 2 of 2096 batches (4166 sequences)
Processing 3 of 2096 batches (4000 sequences)
Processing 4 of 2096 batches (3703 sequences)
Processing 5 of 2096 batches (3571 sequences)
Processing 6 of 2096 batches (3448 sequences)
Processing 7 of 2096 batches (3333 sequences)
Processing 8 of 2096 batches (3225 sequences)
Processing 9 of 2096 batches (3030 sequences)
Processing 10 of 2096 batches (2941 sequences)
Processing 11 of 2096 batches (2857 sequences)
Processing 12 of 2096 batches (2857 sequences)
Processing 13 of 2096 batches (2777 sequences)
Processing 14 of 2096 batches (2777 sequences)
Processing 15 of 2096 batches (2702 sequences)
Processing 16 of 2096 batches (2631 sequences)
Processing 17 of 2096 batches (2631 sequences)
Processing 18 of 2096 batches (2564 sequences)
Processing 19 of 2096 batches (2500 sequences)
Processing 20 of 2096 batches (2500 

In [ ]:
for row in toy_dataset.iterrows():
    seq = row[1]['sequence']
    print(row)
    print(seq)
    break

In [ ]:
def pad_seq_to_max_len(seq, max_len):
    padding_token = '<pad>'
    seq = seq + padding_token * (max_len - len(seq))
    return seq

We want to pad sequences to all be <= a defined length.

In [ ]:
data = [(row[1]['sequence_name'], pad_seq_to_max_len(row[1].sequence, protein_len)) for row in toy_dataset.iterrows()]

In [ ]:
len(data)

6952

In [ ]:
import tqdm
device = 'cuda'

KEY AREA TO IMPROVE:

Here we save the embeddings in a directory ran one-by-one... This is extremely inefficient given we have 1 million sequences... \

The below script will extract embeddings given a Fasta file efficiently.

python esm/scripts/extract.py esm2_t33_650M_UR50D examples/data/some_proteins.fasta \
    examples/data/some_proteins_emb_esm2 --repr_layers 33 --include per_tok

You can write Fasta files from pandas dataframes easily. Here's an example: https://bootstrappers.umassmed.edu/guides/main/r_writeFasta.html

In [ ]:
for i, protein in tqdm.tqdm(enumerate(data)):
    batch_labels, batch_strs, batch_tokens = batch_converter([protein])
    batch_tokens = batch_tokens.to(device)
    with torch.no_grad():
        results = esm_model(batch_tokens, repr_layers=[33], return_contacts=False)
    embeddings = results['representations'][33].detach().cpu().numpy()
    sequence_name = protein[0]
    # Modify all slashes in sequence name to dashes or sees name as directory, perhaps this is a good thing...
    sequence_name = sequence_name.replace('/', '-')
    torch.save(embeddings, f'../data/toy_esm_embeddings/{sequence_name}.pt')


6952it [04:44, 24.40it/s]


In [ ]:
sequence_name = row[1]['sequence_name']
sequence_name = sequence_name.replace('/', '-')

In [ ]:
sequence_name

'S6CDL6_9ACTN-62-159'

In [ ]:
ex = torch.load(f'../data/toy_esm_embeddings/{sequence_name}.pt')

In [ ]:
ex.shape

(1, 202, 1280)

In [ ]:
class ESMFnDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, classes, device='cuda', max_len=200):
        self.dataset = dataset
        self.classes = classes
        self.max_len = max_len
        self.device = device
        self.class_to_idx = {classes[i]: i for i in range(len(classes))}
        self.idx_to_class = {i: classes[i] for i in range(len(classes))}
        self.letter_to_num = {'C': 4, 'D': 3, 'S': 15, 'Q': 5, 'K': 11, 'I': 9,
                'P': 14, 'T': 16, 'F': 13, 'A': 0, 'G': 7, 'H': 8,
                'E': 6, 'L': 10, 'R': 1, 'W': 17, 'V': 19, 
                'N': 2, 'Y': 18, 'M': 12}
        self.num_to_letter = {v:k for k, v in self.letter_to_num.items()}
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        row = self.dataset.iloc[idx]
        sequence_name = row['sequence_name']
        sequence_name = sequence_name.replace('/', '-')
        embeddings = torch.load(f'../data/toy_esm_embeddings/{sequence_name}.pt')
        embeddings = torch.tensor(embeddings, device='cuda', dtype=torch.float32)

        '''Pad embeddings to max_len with zero vector'''
        if embeddings.size(1) < self.max_len:
            B, N, h = embeddings.size()
            pad = torch.zeros((B, self.max_len - embeddings.shape[1], h), device=self.device)
            embeddings = torch.cat((embeddings, pad), dim=1)

        class_idx = torch.tensor(self.class_to_idx[row['family_accession']])
        label = F.one_hot(class_idx, num_classes=len(self.classes))
        return embeddings, label

In [ ]:
dataset = ESMFnDataset(toy_dataset, classes, device)

In [ ]:
esmdl = torch.utils.data.DataLoader(dataset)

In [ ]:
for batch in esmdl:
    print(batch[0][0])
    break

tensor([[[ 0.0243,  0.0271, -0.0837,  ..., -0.2799,  0.1995,  0.0577],
         [-0.0919, -0.0847, -0.0205,  ..., -0.0418,  0.1840, -0.1400],
         [ 0.1741,  0.1002,  0.0078,  ...,  0.0821,  0.0727, -0.0905],
         ...,
         [-0.0306, -0.0665,  0.0843,  ..., -0.0120, -0.0695, -0.1188],
         [-0.0623,  0.0503,  0.0476,  ...,  0.0348, -0.0654, -0.0851],
         [-0.0286,  0.0321,  0.0917,  ..., -0.0120, -0.0180, -0.0668]]],
       device='cuda:0')


In [ ]:
class TransformerEncoder(torch.nn.Module):
    
    def __init__(self, input_size=1280, nheads=8, num_layers=6, device='cuda', classes=2, max_len=202, **kwargs):
        super(TransformerEncoder, self).__init__()
        self.input_size = input_size
        self.classes = classes
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=self.input_size, nhead=nheads, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc1 = torch.nn.Linear(self.input_size, 256)
        self.fc2 = torch.nn.Linear(256, 32)
        self.fc3 = torch.nn.Linear(32 * max_len, self.classes)
        self.softmax = torch.nn.Softmax(dim=-1)
        self.device = device
        
    def forward(self, embedding):
        if len(embedding.size()) > 3 and embedding.size(0) == 1:
            embedding = embedding.squeeze(0)
            assert len(embedding.size()) == 3, 'Embedding has greater than 4 dimensions'
            
        B, N, h = embedding.shape
        hidden = self.transformer_encoder(embedding)
        hidden = self.fc1(hidden)
        hidden = self.fc2(hidden)
        '''Flatten hidden state'''
        hidden = hidden.view(B, -1)
        hidden = self.fc3(hidden)
        hidden = self.softmax(hidden)
        return hidden

In [ ]:
model = TransformerEncoder(input_size=1280, nheads=8, num_layers=6, device=device, classes=len(classes)).to(device)

In [ ]:
len(classes)

2

In [ ]:
for batch in esmdl:
    embedding, label = batch
    pred = model(embedding)
    print(pred.shape) # shape is [batch_size, number of classes]
    break

torch.Size([1, 2])


# Learning to Split

Having trouble running Learning to Split right now because of environment issues.
Regardless you can focus on getting ESMFnDataset to work will all sequences.

Honestly, I don't know if it's possible to run both this Transformer model on 1 million sequences nor Learning to Split but it seems like if there's anyone who can figure it out, it's you.

In [ ]:
import ls

: 

In [ ]:
# python scripts/extract.py esm2_t33_650M_UR50D examples/data/some_proteins.fasta \
#   examples/data/some_proteins_emb_esm2 --repr_layers 33 --include per_tok

: 